# Estimation of Physician Fees Adjusted for Demographic Data

#### Background
One day, there comes a new patient (either from other states with unknown insurance coverage or uninsured) to visit your clinic. The patient wants to see a phsician and order several medical services. If you were the physician, what would you do? Reject him because the patient was not covered by a valid insurance. However, this could cause trouble for your clinic or yourself if the patient could not get in-time cure. If you accept the case, then you might think how much you you are going to charge for per medical service? You don't want to get accused because of charging too much. Neither do you want to lose your job/get bothered by monthly utility bill. Here comes our **Physician Fee  Estimation Project**. 

## 1 The Goal
We simplify the process of complex fee formula by CMS and offers an estimation of the service fee based on proportion and costs of current insurance types. That is to say, physicians get paid for a particular procedure that they perform for their patients depending on the proportion of patients with each insurance type.  Each insurance type, such as medicaid, medicare, and private insurance have different prices for each procedure. these prices are published as fee schedules. So if we want to determine the price a physician would be willing to accept from a new insurance or an uninsured patient, we need to calculate that weighted fee based on fee schedules for those known insurance coverages. The weights are determined as the proportion of the population, such as medicare recipients or medicaid recipients,  in each state or county. Our analysis involves getting the fee schedules, and proportions of each population within different states and counties.

#### Assumptions
1. The gap between actual cost/expense and fee schedule for a specific medical procedure service can be ignored.
2. The fees schedule for private providers are temporarily same as MEDICARE because of their confidentiality.
3. The portion of private providers is seen as the population who is not enrolled in MEDICAID or MEDICARE because of their confidentiality.
4. Given non-facility and facility fees from Medicaid and Medicare insurance, we suppose that physicians would like to choose the max one, i.e. non-facility fee amount.

##### Additional References (optional-can remove later)
- [Markdown Github helper for writing/formatting syntax](https://help.github.com/en/github/writing-on-github/basic-writing-and-formatting-syntax)
- [LeTex for scientific formulas](https://www.math.ubc.ca/~pwalls/math-python/jupyter/latex/)
- [How to handle SettingWithCopyWarning](https://www.dataquest.io/blog/settingwithcopywarning/)
- [CPT Code](https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Part-B-National-Summary-Data-File/Overview)
The data sets are summarized by meaningful ***Healthcare Common Procedure Coding/Current Procedural Terminology, (HCPC/CPT)***, code ranges. Brief descriptions for the code ranges and modifiers are provided in the readme file. The data set name contains the year followed by a five character sequence that is the HCPC/CPT code. This HCPC/CPT code corresponds to the first HCPC/CPT in the selected code range of disciplines.
- [HCPCT Codeset](https://www.cms.gov/Medicare/Coding/HCPCSReleaseCodeSets/Alpha-Numeric-HCPCS)These files contain the Level II alphanumeric HCPCS procedure and modifier codes, their long and short descriptions, and applicable Medicare administrative, coverage and pricing data

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import os
import re

import string
import matplotlib.pyplot as plotter
import matplotlib.pyplot as plt

from collections import Counter
from time import time

# 2 Input
1. **All states population and Medicaid/Medicare Enrollment**<br>
    - The all states population data is obtained from [***United States Census Bureau***](https://www2.census.gov/programs-surveys/popest/datasets/), the latest one is [***2018 all state population estimation***](https://www2.census.gov/programs-surveys/popest/datasets/2010-2018/counties/totals/co-est2018-alldata.csv)
    - The Medicaid enrollment data by state is obtained from [***Medicaid and CHIP Enrollment data***](https://data.medicaid.gov/Enrollment/2018-12-Updated-applications-eligibility-determina/gy72-q4z9/data)
    - The Medicare enrollment data by state and county is obtained from [***CMS Public Use File***](https://www.cms.gov/files/zip/statecounty-table-all-beneficiaries.zip)
2. **Medicaid Physician Fee Schedule For Connecticut/Indiana/New York**<br>
    - Connecticut Medicaid Physcian Fee Schedule is obtained from [***Provider Fee Schedule Portal***](https://www.ctdssmap.com/CTPortal/Provider/ProviderFeeScheduleDownload/tabid/54/Default.aspx)
    - Indiana Medicaid Physician Fee Schedule is from [***IHCP Fee Schedule***](http://provider.indianamedicaid.com/ihcp/Publications/MaxFee/fee_home.asp)
    - NewYork Medicaid Physician Fee Schedule is from [***MED Comply***](https://med-comply.com/NY-Medicaid-Fee-Schedule)
3. **Medicare Physcian Fee Schedule** <br>
    - As we know Medicare is a Federal program, it is organized and published through the search tool [***CMS Physician Fee Schedule Search***](https://www.cms.gov/apps/physician-fee-schedule/search/search-criteria.aspx). This is its related [***specification***](https://www.cms.gov/apps/physician-fee-schedule/help/How_to_MPFS_Booklet_ICN901344.pdf) for the searchable tool. Here is an example on how to get Medicare Physician Fee Schedule datasets for our interest:
        - Year: the fiscal year 2018
        - HCSPC Code Range: the list is from what we got from Medicaid PFS
        - Locality: it depends on the state that we choose
<img src="images/IN_medicare_pfs.png">

In [2]:
%%time
# Population Data from U.S. Census Bureau
df_pop_alldata_2018 = pd.read_csv("data/FY2018_pop_est_alldata.csv", usecols=[x for x in range(3,18)], 
                                  index_col = None, encoding="ISO-8859-1")

#*** State Medicare / Medicaid Enrollment ***#
# Y2018 Medicaid Enrollment Data By State
df_mdcaid_2018Bs = pd.read_csv("data/FY2018_Medicaid_enrollment_data_By_State.csv", encoding="ISO-8859-1")
# Y2018 Medicare Enrollment Data By State and County
df_mdcare_2018Bsc = pd.read_excel("data/FY2018_Medicare_By_State_County.xlsx", 
                                  sheet_name="State_county 2018", header=1, index_col=None)
#*** County Medicare / Medicaid Enrollment ***#
df_IN_mdcaid_2018Bc = pd.read_excel("data/Indiana/FY2018_IN_Medicaid_Enrollment_By_County.xlsx", 
                                    sheet_name="County", header=8, nrows=95, index_col=None)

df_NY_mdcaid_2018Bc = pd.read_excel("data/NewYork/FY2018_NY_medicaid_enroll_by_county.xls", 
                                   sheet_name="Medicaid Managed Care", header=5, index_col=None)

#*** Medicaid physician fee schedule ***#
# Connecticut #
df_CT_mdcaid_ASCPFS_2018 = pd.read_csv("data/Connecticut/FY2018_CT_Medicaid_PFS_casc_24.csv", 
                                       header=2, index_col=None)
# Indiana #
df_IN_mdcaid_OPFS_2018 = pd.read_excel("data/Indiana/FY2018_IN_Medicaid_Outpatient_Fee_Schedule.xlsx", 
                                       sheet_name="Tab 3 - Fee Schedule", header=16, index_col=None, usecols=None)
# New York #
df_NY_mdcaid_MPFS_2018 = pd.read_excel("data/NewYork/FY2018_NY_Physician_Manual_Fee_Schedule_Sect5.xls", 
                                       sheet_name="PHY SURG FS JAN 2020", header=2, index_col=None, usecols=None)
#*** Medicare physician fee schedule ***#
# Indiana #
df_IN_mdcare_pfs_2018 = pd.read_csv("data/Indiana/FY2018_IN_Medicare_PFSExport.csv", 
                                    header=0, index_col=None, usecols=[0, 2, 5])
# Connecticut #
df_CT_mdcare_pfs_2018 = pd.read_csv("data/Connecticut/FY2018_CT_Medicare_PFSExport.csv", 
                                    header=0, index_col=None, usecols=[0, 2, 5])
# New York #
df_NY_mdcare_pfs_2018 = pd.read_csv("data/NewYork/FY2018_NY_medicare_PFSExport.csv", 
                                    header=0, index_col=None, usecols=[0, 2, 5])

CPU times: user 9.85 s, sys: 177 ms, total: 10 s
Wall time: 10.4 s


### 2.1 Get to know our population and insurance enrollment data
Let's do EDA-Exploratory Data Analysis on the datasets.
#### 2.1.1 EDA on the population data

In [3]:
#Population
df_pop_alldata_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Data columns (total 15 columns):
STATE                3193 non-null int64
COUNTY               3193 non-null int64
STNAME               3193 non-null object
CTYNAME              3193 non-null object
CENSUS2010POP        3193 non-null int64
ESTIMATESBASE2010    3193 non-null int64
POPESTIMATE2010      3193 non-null int64
POPESTIMATE2011      3193 non-null int64
POPESTIMATE2012      3193 non-null int64
POPESTIMATE2013      3193 non-null int64
POPESTIMATE2014      3193 non-null int64
POPESTIMATE2015      3193 non-null int64
POPESTIMATE2016      3193 non-null int64
POPESTIMATE2017      3193 non-null int64
POPESTIMATE2018      3193 non-null int64
dtypes: int64(13), object(2)
memory usage: 374.3+ KB


In [4]:
df_pop_alldata_2018.head()

,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
0,1,0,Alabama,Alabama,4779736,4780138,4785448,4798834,4815564,4830460,4842481,4853160,4864745,4875120,4887871
1,1,1,Alabama,Autauga County,54571,54574,54754,55208,54936,54713,54876,54838,55242,55443,55601
2,1,3,Alabama,Baldwin County,182265,182264,183111,186540,190143,194886,199189,202995,207712,212619,218022
3,1,5,Alabama,Barbour County,27457,27457,27330,27350,27174,26944,26758,26294,25819,25158,24881
4,1,7,Alabama,Bibb County,22915,22920,22872,22747,22664,22516,22541,22562,22576,22555,22400


**Granularity**

When county = 0, the row of record stands for whole state's population counts from year 2010-2018. Non-zero county rows means each county's population counts from year 2010 - 2018.

In [5]:
df_pop_alldata_2018["CTYNAME"] = df_pop_alldata_2018["CTYNAME"].apply(lambda x:x[:x.find("County")].strip())
df_POP_alldata_2018Bs = df_pop_alldata_2018[df_pop_alldata_2018["COUNTY"] == 0][["STNAME", "CTYNAME", "POPESTIMATE2018"]]
df_POP_alldata_2018Bc = df_pop_alldata_2018[df_pop_alldata_2018["COUNTY"] != 0][["STNAME", "CTYNAME", "POPESTIMATE2018"]]

In [6]:
df_POP_alldata_2018Bs.head()

,STNAME,CTYNAME,POPESTIMATE2018
0,Alabama,Alabam,4887871
68,Alaska,Alask,737438
98,Arizona,Arizon,7171646
114,Arkansas,Arkansa,3013825
190,California,Californi,39557045


In [7]:
df_POP_alldata_2018Bc.head()

,STNAME,CTYNAME,POPESTIMATE2018
1,Alabama,Autauga,55601
2,Alabama,Baldwin,218022
3,Alabama,Barbour,24881
4,Alabama,Bibb,22400
5,Alabama,Blount,57840


#### 2.1.2 Get to know our state Medicaid enrollment data
Let's do some EDA tricks on the dataset.

In [8]:
#Medicaid State Enrollment
df_mdcaid_2018Bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 29 columns):
State Abbreviation                                                                              51 non-null object
State Name                                                                                      51 non-null object
Report Date                                                                                     51 non-null object
State Expanded Medicaid                                                                         51 non-null object
Preliminary_Updated                                                                             51 non-null object
Final_Report                                                                                    51 non-null object
New Applications Submitted to Medicaid and CHIP Agencies                                        48 non-null float64
New Applications Submitted to Medicaid and CHIP Agencies â footnotes                     

In [9]:
df_mdcaid_2018Bs.head()

,State Abbreviation,State Name,Report Date,State Expanded Medicaid,Preliminary_Updated,Final_Report,New Applications Submitted to Medicaid and CHIP Agencies,New Applications Submitted to Medicaid and CHIP Agencies â footnotes,Applications for Financial Assistance Submitted to the State Based Marketplace,Applications for Financial Assistance Submitted to the State Based Marketplace â footnotes,...,Medicaid and CHIP Child Enrollment - footnotes,Total Medicaid and CHIP Enrollment,Total Medicaid and CHIP Enrollment - footnotes,Latitude,Longitude,New Georeferenced Column,Total Medicaid Enrollment,Total Medicaid Enrollment - footnotes,Total CHIP Enrollment,Total CHIP Enrollment - footnotes
0,TN,Tennessee,12/01/2018,N,U,Y,NaN,NaN,NaN,NaN,...,NaN,1396302,NaN,41.6772,-71.5101,"(41.6772, -71.5101)",1342027,NaN,54275,NaN
1,ID,Idaho,12/01/2018,N,U,Y,14639.0,NaN,NaN,NaN,...,NaN,280570,NaN,42.0046,-93.2140,"(42.0046, -93.214)",256565,NaN,24005,NaN
2,MA,Massachusetts,12/01/2018,Y,U,Y,16180.0,NaN,4462.0,NaN,...,NaN,1598878,NaN,31.1801,-91.8749,"(31.1801, -91.8749)",1407486,NaN,191392,NaN
3,NM,New Mexico,12/01/2018,Y,U,Y,10579.0,NaN,NaN,NaN,...,NaN,728327,NaN,43.4108,-71.5653,"(43.4108, -71.5653)",691223,NaN,37104,NaN
4,HI,Hawaii,12/01/2018,Y,U,Y,4822.0,NaN,NaN,NaN,...,NaN,331075,NaN,32.9866,-83.6487,"(32.9866, -83.6487)",305872,NaN,25203,NaN


Each row represents the statistic in one state. We need to select and convert the label names for some attributes to better visualize and organize the data.

In [10]:
df_MDcaid_2018Bs = df_mdcaid_2018Bs[["State Abbreviation", "State Name", "Total Medicaid and CHIP Enrollment"]].copy()
df_MDcaid_2018Bs.rename(columns={"State Name":"STNAME", "State Abbreviation":"STATE", "Total Medicaid and CHIP Enrollment":"MDCAID_CNT_2018BS"}, inplace=True)
df_MDcaid_2018Bs.head()

,STATE,STNAME,MDCAID_CNT_2018BS
0,TN,Tennessee,1396302
1,ID,Idaho,280570
2,MA,Massachusetts,1598878
3,NM,New Mexico,728327
4,HI,Hawaii,331075


#### 2.1.3 Get to know our state-county Medicare enrollment data
Let's do some EDA tricks on the dataset.

In [11]:
df_mdcare_2018Bsc.head()

,State,County,State and County FIPS Code,Beneficiaries with Part A and Part B,FFS Beneficiaries,MA Beneficiaries,MA Participation Rate,Average Age,Percent Female,Percent Male,...,PQI11 Bacterial Pneumonia Admission Rate (age < 65),PQI11 Bacterial Pneumonia Admission Rate (age 65-74),PQI11 Bacterial Pneumonia Admission Rate (age 75+),PQI12 UTI Admission Rate (age < 65),PQI12 UTI Admission Rate (age 65-74),PQI12 UTI Admission Rate (age 75+),PQI15 Asthma in Younger Adults Admission Rate (age < 40),PQI16 Lower Extremity Amputation Admission Rate (age < 65),PQI16 Lower Extremity Amputation Admission Rate (age 65-74),PQI16 Lower Extremity Amputation Admission Rate (age 75+)
0,Na,NATIONAL TOTAL,NaN,56031636,33499472,22532164,40.21 %,72,54.67 %,45.33 %,...,497,344,1005,292,219,943,159,238,69,58
1,AK,STATE TOTAL,NaN,86462,84714,1748,2.02 %,71,50.41 %,49.59 %,...,211,203,728,140,99,499,*,193,35,70
2,AK,Aleutians East,2013.0,*,117,*,*,72,47.01 %,52.99 %,...,,,,,,,,,,
3,AK,Aleutians West,2016.0,*,135,*,*,71,46.67 %,53.33 %,...,,,,,,,,,,
4,AK,Anchorage,2020.0,32227,31503,724,2.25 %,71,52.76 %,47.24 %,...,,,,,,,,,,


In [12]:
df_mdcare_2018Bsc.iloc[:,0:6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250 entries, 0 to 3249
Data columns (total 6 columns):
State                                   3250 non-null object
County                                  3250 non-null object
State and County FIPS Code              3194 non-null float64
Beneficiaries with Part A and Part B    3250 non-null object
FFS Beneficiaries                       3250 non-null object
MA Beneficiaries                        3250 non-null object
dtypes: float64(1), object(5)
memory usage: 152.4+ KB


**Granularity**
1. First, we can get the first row means the national total counts of Medicare beneficiaries.
2. Second, for each state, the row statistics starts from state total counts of Medicare beneficiaries following by each county
3. We could see that there are invalid and missing values in total counts of medicare beneficiaries, such as **"*"**
4. We figure out the relationship of column **"Beneficiaries with Part A and Part B" is the sum of "FFS Beneficiaries" and "MA Beneficiaries"**.

In [13]:
# DataFrame Column Labels Formatter
df_MDcare_2018Bsc = df_mdcare_2018Bsc[["State", "County", "Beneficiaries with Part A and Part B", "FFS Beneficiaries", "MA Beneficiaries"]].copy()
df_MDcare_2018Bsc.rename(columns = {"State":"STATE", "County":"CTYNAME",
                                   "Beneficiaries with Part A and Part B":"MDCARE_CNT_2018",
                                   "FFS Beneficiaries":"FFS_CNT",
                                   "MA Beneficiaries":"MA_CNT"}, inplace=True)
df_MDcare_2018Bsc["FFS_CNT"] = df_MDcare_2018Bsc["FFS_CNT"].apply(lambda x: int(x) if x != "*" else 0)
df_MDcare_2018Bsc["MA_CNT"] = df_MDcare_2018Bsc["MA_CNT"].apply(lambda x: int(x) if x != "*" else 0)
df_MDcare_2018Bsc["MDCARE_CNT_2018"] = df_MDcare_2018Bsc["MDCARE_CNT_2018"].apply(lambda x: int(x) if x != "*" else 0)
df_MDcare_2018Bsc["MDCARE_CNT_2018"] = df_MDcare_2018Bsc["FFS_CNT"] + df_MDcare_2018Bsc["MA_CNT"]
df_MDcare_2018Bsc.head()

,STATE,CTYNAME,MDCARE_CNT_2018,FFS_CNT,MA_CNT
0,Na,NATIONAL TOTAL,56031636,33499472,22532164
1,AK,STATE TOTAL,86462,84714,1748
2,AK,Aleutians East,117,117,0
3,AK,Aleutians West,135,135,0
4,AK,Anchorage,32227,31503,724


In [14]:
df_MDcare_2018Bsc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250 entries, 0 to 3249
Data columns (total 5 columns):
STATE              3250 non-null object
CTYNAME            3250 non-null object
MDCARE_CNT_2018    3250 non-null int64
FFS_CNT            3250 non-null int64
MA_CNT             3250 non-null int64
dtypes: int64(3), object(2)
memory usage: 127.0+ KB


5. We then get the Medicare enroll separately by state and by county through CTYNAME is "STATE TOTAL" or not

In [15]:
# PICK up state level statistics
df_mdcare_2018Bs = df_MDcare_2018Bsc[df_MDcare_2018Bsc["CTYNAME"] == "STATE TOTAL"].copy()
df_MDcare_2018Bs = df_mdcare_2018Bs[["STATE", "MDCARE_CNT_2018"]].copy()
df_mdcare_2018Bc = df_MDcare_2018Bsc[~df_MDcare_2018Bsc["CTYNAME"].isin(["STATE TOTAL", "NATIONAL TOTAL"])].copy()
df_MDcare_2018Bc = df_mdcare_2018Bc[["STATE", "CTYNAME", "MDCARE_CNT_2018"]]

In [16]:
df_MDcare_2018Bs.head()

,STATE,MDCARE_CNT_2018
1,AK,86462
32,AL,985296
101,AR,602253
178,AZ,1199206
195,CA,5608325


In [17]:
df_MDcare_2018Bc.head()

,STATE,CTYNAME,MDCARE_CNT_2018
2,AK,Aleutians East,117
3,AK,Aleutians West,135
4,AK,Anchorage,32227
5,AK,Bethel,1224
6,AK,Bristol Bay,116


### 2.2 Get the county-level weights for different types of insurance physician fee
As we have already collected the census population, medicare enrollment population by county level. However, the county level enrollment of medicaid is not published in the CMS(Centers for Medicare and Medicaid Service). In order to get the county level statistics, we have to dive into the local government health administration website. For generalization, we take below states (Indiana, Connecticut and New York) as our research data.
#### 2.2.1 EDA on Indiana County Medicaid Enrollment
- [Medicaid county level](https://www.in.gov/fssa/ompp/4881.htm)

In [18]:
# Indiana county level by state
df_IN_mdcaid_2018Bc.head()

,Anthem,CareSource,MDwise,MHS,Total,Anthem.1,MHS.1,Total.1,Anthem.2,CareSource.1,MDwise.1,MHS.2,Unassigned1,Total.2,Total.3,County Total
01-ADAMS,555.0,237.0,545.0,578.0,1915.0,180.0,102.0,282.0,491.0,132.0,290.0,295.0,18.0,1226.0,1127.0,4550
02-ALLEN,13119.0,3622.0,14646.0,5465.0,36852.0,3471.0,2020.0,5491.0,8157.0,2441.0,6782.0,3161.0,1239.0,21780.0,16594.0,80717
03-BARTHOLOMEW,894.0,394.0,1801.0,2663.0,5752.0,350.0,404.0,754.0,1218.0,314.0,989.0,1116.0,309.0,3946.0,3256.0,13708
04-BENTON,155.0,108.0,501.0,188.0,952.0,52.0,53.0,105.0,161.0,64.0,246.0,114.0,15.0,600.0,447.0,2104
05-BLACKFORD,315.0,123.0,563.0,255.0,1256.0,96.0,86.0,182.0,335.0,80.0,392.0,128.0,3.0,938.0,733.0,3109


**Granularity**
1. The index is the county name as well as their id
2. The column "County Total" contains all the Medicaid enrollment counts of per county

In [19]:
df_IN_m1_tmp = df_IN_mdcaid_2018Bc.reset_index()
df_IN_m1_tmp.rename(columns={"index":"COUNTY_ID_NAME", "County Total":"MDCAID_CNT"}, inplace=True)
df_IN_MDcaid_2018Bc = df_IN_m1_tmp[["COUNTY_ID_NAME", "MDCAID_CNT"]].copy()
df_IN_MDcaid_2018Bc[["COUNTY_ID", "CTYNAME"]] = df_IN_MDcaid_2018Bc.COUNTY_ID_NAME.str.split("-", expand=True)
df_IN_MDcaid_2018Bc.CTYNAME = df_IN_MDcaid_2018Bc.CTYNAME.apply(lambda x: x.lower().strip().capitalize())
df_IN_MDcaid_2018Bc = df_IN_MDcaid_2018Bc[["CTYNAME", "MDCAID_CNT"]].copy()
df_IN_MDcaid_2018Bc.head()

,CTYNAME,MDCAID_CNT
0,Adams,4550
1,Allen,80717
2,Bartholomew,13708
3,Benton,2104
4,Blackford,3109


#### 2.2.2 Compute the county Medicaid Enrollment in Connecticut
To get the state Connecticut county level data - enrollment population of medicaid, we searched the local government [Connecticut medicaid monthly enrollment reports](https://portal.ct.gov/DSS/ITS/DSS-HealthIT/Business-Intelligence-and-DSS-HealthIT/Data-and-Program-Reports). However, the website contains only township medicaid enrollment data as below: 
- [DSS Township Counts - by Program - CY 2018](https://data.ct.gov/Health-and-Human-Services/DSS-Township-Counts-by-Program-CY-2018/n5xw-nk45)
Connecticut’s Medicaid program is called HUSKY Health, and it is broken into several categories (enrollment and cost data based on a report published in 2018).

*To download Towns and Counties relationship table in Connecticut. Steps: download the html in local pc, then open it with excel. You can generate the table in the excel as in the html😀[Towns and Counties List in Connecticut](https://ctstatelibrary.org/cttowns/counties)*

In [20]:
df_CT_DSS_2018Bt = pd.read_csv("data/Connecticut/FY2018_CT_Medicaid_DSS_Township_Counts.csv", 
                                   header=0, index_col=None, usecols=None)
df_CT_Town_County = pd.read_excel("data/Connecticut/FY2018_CT_Towns_and_Counties.xlsx",
                                 sheet_name="townslist", header=0, index_col=None)

In [21]:
df_CT_DSS_2018Bt.head()

,township,program,count
0,Andover,CADAP,0
1,Andover,CHCPE,0
2,Andover,CHIP,32
3,Andover,Medicaid,461
4,Andover,MSP,117


To include only counts from Medicaid/CHIP program in DSS table of Connecticut state, we could use Groupby property of the dataframe to compute counties with the township medicaid enrollment to get our county level enrollment.

In [22]:
df_CT_mdcaid_2018Bt = df_CT_DSS_2018Bt[df_CT_DSS_2018Bt["program"].isin(["Medicaid", "CHIP"])].copy()
# Use .loc(label)/.iloc(int) to avoid SettingWithCopyWarning in Pandas
df_CT_mdcaid_2018Bt["count"] = df_CT_mdcaid_2018Bt["count"].apply(lambda x:x.replace(r',', ''))
df_CT_mdcaid_2018Bt["count"] = df_CT_mdcaid_2018Bt["count"].apply(lambda x:int(x.replace(r'[^-+\d.]', '')))
df_CT_Mdcaid_2018Bt = df_CT_mdcaid_2018Bt.groupby(["township"])["count"].sum().reset_index()
df_CT_Mdcaid_2018Bt.rename(columns={"count":"MDCAID_CNT"}, inplace=True)
df_CT_Mdcaid_2018Bt.head()

,township,MDCAID_CNT
0,Andover,493
1,Ansonia,8043
2,Ashford,1062
3,Avon,1660
4,Barkhamsted,560


In [23]:
df_CT_Town_County.head()

,Town name,County,Year Established,Parent Town,History of incorporation
0,Andover,Tolland,1848,"Coventry, Hebron","May 18, 1848; taken from Hebron and Coventry"
1,Ansonia,New Haven,1889,Derby,"April, 1889; taken from Derby"
2,Ashford,Windham,1714,—-,"Oct., 1714"
3,Avon,Hartford,1830,Farmington,"May, 1830; taken from Farmington"
4,Barkhamsted,Litchfield,1779,—-,"Oct., 1779"


In [24]:
df_CT_tc = df_CT_Town_County.rename(columns = {"Town name":"township", "County":"CTYNAME"})
df_CT_CTY = pd.merge(df_CT_Mdcaid_2018Bt, df_CT_tc, on="township")
df_CT_MDcaid_2018Bc = df_CT_CTY.groupby(["CTYNAME"])["MDCAID_CNT"].sum().reset_index()
df_CT_MDcaid_2018Bc.head()

,CTYNAME,MDCAID_CNT
0,Fairfield,239342
1,Hartford,295583
2,Litchfield,44492
3,Middlesex,34245
4,New Haven,297593


#### 2.2.3 EDA on NewYork County Medicaid Enrollment

In [25]:
df_NY_mdcaid_2018Bc.head(10)

,County,Plan Name,Roster Enrollment,Roster Enrollment.1,Roster Enrollment.2,Enrollment,Unnamed: 6
0,Albany,TOTALS:,4152.0,822.0,3405.0,36066.0,44445.0
1,Mandatory,Capital District Physicians Health Plan,2574.0,467.0,2223.0,20249.0,25513.0
2,Eff. Oct 1997,Fidelis Care,1063.0,249.0,834.0,11645.0,13791.0
3,NaN,MVP Health Plan,285.0,50.0,96.0,2184.0,2615.0
4,NaN,United Healthcare Plan of NY,91.0,20.0,74.0,749.0,934.0
5,NaN,Wellcare of New York,139.0,36.0,178.0,1239.0,1592.0
6,Allegany,TOTALS:,730.0,247.0,615.0,6528.0,8120.0
7,Mandatory,Fidelis Care,321.0,122.0,210.0,3340.0,3993.0
8,Eff. Feb 2007,HealthNow,169.0,55.0,222.0,1271.0,1717.0
9,NaN,YourCare Health Plan,240.0,70.0,183.0,1917.0,2410.0


In [26]:
df_NY_mdcaid_2018Bc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 7 columns):
County                 179 non-null object
Plan Name              274 non-null object
Roster Enrollment      273 non-null float64
Roster Enrollment.1    273 non-null float64
Roster Enrollment.2    273 non-null float64
Enrollment             273 non-null float64
Unnamed: 6             273 non-null float64
dtypes: float64(5), object(2)
memory usage: 15.2+ KB


**Granularity**
1. The County columns of N.Y. medicaid enrollment data contains invalid data. We can filter it out by using Plan Name = "TOTALS:"
2. The total enrolled column is our total medicaid enrollment counts which is labeled by "Unnamed:6". But the column labels are unreadable, we need to fix it.
3. The type of the counts are float which is computable. But to make it more accurate and understandable, we can convert it to integer.

In [27]:
df_NY_m1_2018Bc = df_NY_mdcaid_2018Bc.rename(columns={"County":"CTYNAME", "Unnamed: 6":"MDCAID_CNT"}).copy()
df_NY_m1_2018Bc = df_NY_m1_2018Bc[df_NY_m1_2018Bc["Plan Name"] == "TOTALS:"]
df_NY_m1_2018Bc["MDCAID_CNT"] = df_NY_m1_2018Bc["MDCAID_CNT"].apply(int)
df_NY_MDcaid_2018Bc = df_NY_m1_2018Bc[["CTYNAME", "MDCAID_CNT"]]
df_NY_MDcaid_2018Bc.head()

,CTYNAME,MDCAID_CNT
0,Albany,44445
6,Allegany,8120
10,Broome,34689
15,Cattaraugus,14507
19,Cayuga,12481


### 2.3 Get to know our PFS data
#### 2.3.1 Indiana PFS

In [28]:
# Indiana's Medicaid Physician Fee Schedule
df_IN_mdcaid_OPFS_2018.head()

,Proc Code,Description,IPO CODE,PA,Cov,Pricing,HAF Exempt?,Fee Sched Amt,Manual Method,Price Effective,ASC
0,10004,Fna bx w/o img gdn ea addl,No,No,Yes,NONE,No,NaN,NaN,2019-01-01 00:00:00,NaN
1,10005,Fna bx w/us gdn 1st les,No,No,Yes,PC,No,579.34,NaN,2019-01-01 00:00:00,NaN
2,10006,Fna bx w/us gdn ea addl,No,No,Yes,NONE,No,NaN,NaN,2019-01-01 00:00:00,NaN
3,10007,Fna bx w/fluor gdn 1st les,No,No,Yes,PC,No,579.34,NaN,2019-01-01 00:00:00,NaN
4,10008,Fna bx w/fluor gdn ea addl,No,No,Yes,NONE,No,NaN,NaN,2019-01-01 00:00:00,NaN


In [29]:
df_IN_mdcaid_OPFS_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16534 entries, 0 to 16533
Data columns (total 11 columns):
Proc Code          16534 non-null object
Description        16534 non-null object
IPO CODE           16534 non-null object
PA                 16530 non-null object
Cov                16534 non-null object
Pricing            12615 non-null object
HAF Exempt?        12821 non-null object
Fee Sched Amt      3535 non-null object
Manual Method      397 non-null object
Price Effective    12822 non-null object
ASC                3988 non-null object
dtypes: object(11)
memory usage: 1.4+ MB


**Granularity**
1. Each row stands for a fee schedule of a procedure service noted by procedure code/HCPCT code
2. To map the temporality of our research timeline, we will only include procedure code with its price which is effective in year 2018.
3. As we can see there are missing values in column "Fee Sched Amt", we handled it by dropping those rows; Meanwhile, we can convert the type of fee amount from object to float. 

In [30]:
df_IN_m1OPFS_2018 = df_IN_mdcaid_OPFS_2018[pd.to_numeric(df_IN_mdcaid_OPFS_2018["Fee Sched Amt"], errors='coerce').notnull()].copy()
df_IN_m1OPFS_2018.rename(columns={"Proc Code":"PROC_CODE", "Fee Sched Amt":"MDCAID_PFS_AMT"}, inplace=True)
df_IN_m1OPFS_2018["PROC_CODE"] = df_IN_m1OPFS_2018["PROC_CODE"].apply(lambda x: str(x).strip())
df_IN_m1OPFS_2018["Price Effective"] = df_IN_m1OPFS_2018["Price Effective"].apply(pd.to_datetime)
df_IN_m1OPFS_2018 = df_IN_m1OPFS_2018[df_IN_m1OPFS_2018["Price Effective"] < pd.to_datetime("1/1/2019")]
df_IN_m1OPFS_2018["MDCAID_PFS_AMT"] = df_IN_m1OPFS_2018["MDCAID_PFS_AMT"].astype(float).round(2)
df_IN_m1OPFS_2018 = df_IN_m1OPFS_2018[["PROC_CODE", "Description", "MDCAID_PFS_AMT"]]
df_IN_m1OPFS_2018.head()

,PROC_CODE,Description,MDCAID_PFS_AMT
11,10030,GUIDE CATHET FLUID DRAINAGE,539.11
12,10035,PERQ DEV SOFT TISS 1ST IMAG,480.64
370,19081,BX BREAST 1ST LESION STRTCTC,702.08
372,19083,BX BREAST 1ST LESION US IMAG,702.08
374,19085,BX BREAST 1ST LESION MR IMAG,702.08


In [31]:
df_IN_mdcare_pfs_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9194 entries, 0 to 9193
Data columns (total 3 columns):
HCPCS CODE            9194 non-null object
SHORT DESCRIPTION     9194 non-null object
NON-FACILITY PRICE    9194 non-null object
dtypes: object(3)
memory usage: 215.6+ KB


**Granularity**
1. Each row stands for a fee schedule of a procedure service noted by procedure code/HCPCT code
2. As we can see the values are string type and has non numeric characters in it. We handled it by replacing the non numeric characters with space; Meanwhile, we can convert the type of fee amount from object to float. 

In [32]:
def handle_medicare_pfs(df_input):
    """
        This function is to preprocess raw medicare physician fee schedule.
        args:
            df_input(dataframe):
        returns:
            df_output(dataframe): the formatted dataframe after preprocessing the procedure code, and amount attribute
    """
    #Medicare Searchable CPT Pricing info for certain CPT
    df_output = df_input.rename(columns={"HCPCS CODE":"PROC_CODE", "NON-FACILITY PRICE":"MDCARE_PFS_AMT"})
    df_output["PROC_CODE"] = df_output["PROC_CODE"].apply(lambda x:str(x).strip())
    #Handle $12,234.56 char formatted cash amount to float type
    df_output["MDCARE_PFS_AMT"] = df_output["MDCARE_PFS_AMT"].str.replace(r',', '')
    df_output["MDCARE_PFS_AMT"] = df_output["MDCARE_PFS_AMT"].str.replace(r'[^-+\d.]', '').astype(float)
    return df_output

In [33]:
df_IN_m2pfs_2018 = handle_medicare_pfs(df_IN_mdcare_pfs_2018)
df_IN_MMPFS_2018 = pd.merge(df_IN_m1OPFS_2018, df_IN_m2pfs_2018, on="PROC_CODE")
df_IN_PFS_2018 = df_IN_MMPFS_2018[["PROC_CODE", "SHORT DESCRIPTION", "MDCAID_PFS_AMT", "MDCARE_PFS_AMT"]].copy()
#Private Provider Fee Amount
df_IN_PFS_2018["PP_PFS_AMT"] = df_IN_PFS_2018["MDCARE_PFS_AMT"]
df_IN_PFS_2018.head()

,PROC_CODE,SHORT DESCRIPTION,MDCAID_PFS_AMT,MDCARE_PFS_AMT,PP_PFS_AMT
0,10030,Guide cathet fluid drainage,539.11,533.82,533.82
1,10035,Perq dev soft tiss 1st imag,480.64,488.31,488.31
2,19081,Bx breast 1st lesion strtctc,702.08,652.03,652.03
3,19083,Bx breast 1st lesion us imag,702.08,634.19,634.19
4,19085,Bx breast 1st lesion mr imag,702.08,948.13,948.13


#### 2.2.2 Connecticut PFS

In [34]:
df_CT_mdcaid_ASCPFS_2018.head()

,Procedure Code,Proc description,Mod1,Mod1 desc,Rate Type,Max Fee,Effective Date,End Date,PA
0,10121,Remove foreign body,,,ASC,446,10/1/08,12/31/99,
1,10180,Complex drainage wound,,,ASC,446,10/1/08,12/31/99,
2,11010,Debride skin at fx site,,,ASC,251.52,10/1/08,12/31/99,
3,11011,Debride skin musc at fx site,,,ASC,251.52,10/1/08,12/31/99,
4,11012,Deb skin bone at fx site,,,ASC,251.52,10/1/08,12/31/99,


In [35]:
df_CT_mdcaid_ASCPFS_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2626 entries, 0 to 2625
Data columns (total 9 columns):
Procedure Code      2626 non-null object
Proc description    2626 non-null object
Mod1                2626 non-null object
Mod1 desc           2626 non-null object
Rate Type           2626 non-null object
Max Fee             2626 non-null object
Effective Date      2626 non-null object
End Date            2626 non-null object
PA                  2626 non-null object
dtypes: object(9)
memory usage: 184.7+ KB


Granularity

1. Each row stands for a fee schedule of a procedure service noted by procedure code/HCPCT code
2. To map the temporality of our research timeline, we will only include procedure code with its price which is effective in year 2018.
3. As we can see there are missing values in column "Max Fee", we handled it by dropping those rows; Meanwhile, we can convert the type of fee amount from object to float.

In [36]:
df_CT_m1ASCPFS_2018 = df_CT_mdcaid_ASCPFS_2018[pd.to_numeric(df_CT_mdcaid_ASCPFS_2018["Max Fee"], errors='coerce').notnull()].copy()
df_CT_m1ASCPFS_2018.rename(columns={"Procedure Code":"PROC_CODE", "Max Fee":"MDCAID_PFS_AMT"}, inplace=True)
df_CT_m1ASCPFS_2018["PROC_CODE"] = df_CT_m1ASCPFS_2018["PROC_CODE"].apply(lambda x: str(x).strip())

df_CT_m1ASCPFS_2018["Effective Date"] = df_CT_m1ASCPFS_2018["Effective Date"].apply(pd.to_datetime)
df_CT_m1ASCPFS_2018 = df_CT_m1ASCPFS_2018[df_CT_m1ASCPFS_2018["Effective Date"] < pd.to_datetime("1/1/2019")]
df_CT_m1ASCPFS_2018["MDCAID_PFS_AMT"] = df_CT_m1ASCPFS_2018["MDCAID_PFS_AMT"].astype(float).round(2)
df_CT_m1ASCPFS_2018 = df_CT_m1ASCPFS_2018[["PROC_CODE", "Proc description", "MDCAID_PFS_AMT"]]
df_CT_m1ASCPFS_2018.head()

,PROC_CODE,Proc description,MDCAID_PFS_AMT
0,10121,Remove foreign body,446.00
1,10180,Complex drainage wound,446.00
2,11010,Debride skin at fx site,251.52
3,11011,Debride skin musc at fx site,251.52
4,11012,Deb skin bone at fx site,251.52


In [37]:
df_CT_mdcare_pfs_2018.head()

,HCPCS CODE,SHORT DESCRIPTION,NON-FACILITY PRICE
0,A4890,Repair/maint cont hemo equip,$0.00
1,D0150,Comprehensve oral evaluation,$0.00
2,D0240,Intraoral occlusal film,$0.00
3,D0250,Extraoral 2d project image,$0.00
4,D0251,Extraoral posterior image,$0.00


In [38]:
df_CT_m2pfs_2018 = handle_medicare_pfs(df_CT_mdcare_pfs_2018)
df_CT_MMPFS_2018 = pd.merge(df_CT_m1ASCPFS_2018, df_CT_m2pfs_2018, on="PROC_CODE")
df_CT_PFS_2018 = df_CT_MMPFS_2018[["PROC_CODE", "SHORT DESCRIPTION", "MDCAID_PFS_AMT", "MDCARE_PFS_AMT"]].copy()
#Private Provider Fee Amount
df_CT_PFS_2018["PP_PFS_AMT"] = df_CT_PFS_2018["MDCARE_PFS_AMT"]
df_CT_PFS_2018.head()

,PROC_CODE,SHORT DESCRIPTION,MDCAID_PFS_AMT,MDCARE_PFS_AMT,PP_PFS_AMT
0,10121,Remove foreign body,446.00,306.48,306.48
1,10180,Complex drainage wound,446.00,277.66,277.66
2,11010,Debride skin at fx site,251.52,568.10,568.10
3,11011,Debride skin musc at fx site,251.52,608.45,608.45
4,11012,Deb skin bone at fx site,251.52,795.03,795.03


#### 2.2.3 New York PFS

In [39]:
df_NY_mdcaid_MPFS_2018.head()

,CODE,DESCRIPTION,NON-FACILITY GLOBAL FEE,FACILITY GLOBAL FEE,PROFESSIONAL COMPONENT FEE,BR,PA,FU DAYS,CHANGE
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, WITHOUT IMAGING",34.12,23.81,NaN,NaN,0,0,NaN
1,10005,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",82.15,39.23,NaN,NaN,0,0,NaN
2,10006,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",38.92,27.04,NaN,NaN,0,0,NaN
3,10007,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",180.07,51.13,NaN,NaN,0,0,NaN
4,10008,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",104.76,33.31,NaN,NaN,0,0,NaN


In [40]:
df_NY_mdcaid_MPFS_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5659 entries, 0 to 5658
Data columns (total 9 columns):
CODE                          5659 non-null object
DESCRIPTION                   5659 non-null object
NON-FACILITY GLOBAL FEE       5380 non-null object
FACILITY GLOBAL FEE           5380 non-null object
PROFESSIONAL COMPONENT FEE    16 non-null float64
BR                            298 non-null object
PA                            5658 non-null object
FU DAYS                       5659 non-null object
CHANGE                        40 non-null object
dtypes: float64(1), object(8)
memory usage: 398.0+ KB


Granularity

1. Each row stands for a fee schedule of a procedure service noted by procedure code/HCPCT code
2. To map the temporality of our research timeline, we will only include procedure code with its price which is effective in year 2018.
3. As we can see there are missing values in column "Max Fee", we handled it by dropping those rows; Meanwhile, we can convert the type of fee amount from object to float.

In [41]:
df_NY_m1MPFS_2018 = df_NY_mdcaid_MPFS_2018[pd.to_numeric(df_NY_mdcaid_MPFS_2018["NON-FACILITY GLOBAL FEE"], errors='coerce').notnull()].copy()
df_NY_m1MPFS_2018.rename(columns={"CODE":"PROC_CODE", "NON-FACILITY GLOBAL FEE":"MDCAID_PFS_AMT"}, inplace=True)
df_NY_m1MPFS_2018["PROC_CODE"] = df_NY_m1MPFS_2018["PROC_CODE"].apply(lambda x: str(x).strip())
df_NY_m1MPFS_2018["MDCAID_PFS_AMT"] = df_NY_m1MPFS_2018["MDCAID_PFS_AMT"].astype(float).round(2)
df_NY_m1MPFS_2018 = df_NY_m1MPFS_2018[["PROC_CODE", "DESCRIPTION", "MDCAID_PFS_AMT"]]
df_NY_m1MPFS_2018.head()

,PROC_CODE,DESCRIPTION,MDCAID_PFS_AMT
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, WITHOUT IMAGING",34.12
1,10005,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",82.15
2,10006,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",38.92
3,10007,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",180.07
4,10008,"FINE NEEDLE ASPIRATION BIOPSY, INCLUDING",104.76


In [42]:
df_NY_mdcare_pfs_2018.head()

,HCPCS CODE,SHORT DESCRIPTION,NON-FACILITY PRICE
0,10021,Fna w/o image,$118.79
1,10022,Fna w/image,$137.42
2,10030,Guide cathet fluid drainage,$550.32
3,10035,Perq dev soft tiss 1st imag,$503.96
4,10036,Perq dev soft tiss add imag,$441.81


In [43]:
df_NY_mdcare_pfs_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8933 entries, 0 to 8932
Data columns (total 3 columns):
HCPCS CODE            8933 non-null int64
SHORT DESCRIPTION     8933 non-null object
NON-FACILITY PRICE    8933 non-null object
dtypes: int64(1), object(2)
memory usage: 209.4+ KB


In [44]:
df_NY_m2pfs_2018 = handle_medicare_pfs(df_NY_mdcare_pfs_2018)
df_NY_MMPFS_2018 = pd.merge(df_NY_m1MPFS_2018, df_NY_m2pfs_2018, on="PROC_CODE")
df_NY_PFS_2018 = df_NY_MMPFS_2018[["PROC_CODE", "SHORT DESCRIPTION", "MDCAID_PFS_AMT", "MDCARE_PFS_AMT"]].copy()
#Private Provider Fee Amount
df_NY_PFS_2018["PP_PFS_AMT"] = df_NY_PFS_2018["MDCARE_PFS_AMT"]
df_NY_PFS_2018.head()

,PROC_CODE,SHORT DESCRIPTION,MDCAID_PFS_AMT,MDCARE_PFS_AMT,PP_PFS_AMT
0,10021,Fna w/o image,81.61,118.79,118.79
1,10030,Guide cathet fluid drainage,504.97,550.32,550.32
2,10035,Perq dev soft tiss 1st imag,349.63,503.96,503.96
3,10036,Perq dev soft tiss add imag,304.15,441.81,441.81
4,10040,Acne surgery,54.25,107.02,107.02


## 3 Calculate the weights
We could obtain our weighting coefficients based on the fraction of each insurance enrollment of population in state/county
### 3.1 Calculate our state-level weights
We estimate the state-level weights = each insurance enrollment in state / our state whole population

In [45]:
df_POPM1_2018Bs = pd.merge(df_POP_alldata_2018Bs, df_MDcaid_2018Bs, on="STNAME")
df_POPM2_2018Bs = pd.merge(df_POPM1_2018Bs, df_MDcare_2018Bs, on="STATE")
df_POPMM_2018Bs = df_POPM2_2018Bs[["STATE", "STNAME", "POPESTIMATE2018", "MDCAID_CNT_2018BS", "MDCARE_CNT_2018"]].copy()
# Choose "TOTAL MEDICAID AND CHIP ENROLLMENT" as medicaid total
df_POPMM_2018Bs["MDCAID_RATE_2018BS"] = df_POPMM_2018Bs["MDCAID_CNT_2018BS"] / df_POPMM_2018Bs["POPESTIMATE2018"]
df_POPMM_2018Bs["MDCARE_RATE_2018BS"] = df_POPMM_2018Bs["MDCARE_CNT_2018"] / df_POPMM_2018Bs["POPESTIMATE2018"]
df_POPMM_2018Bs["PRIVATE_RATE_2018BS"] = 1 - (df_POPMM_2018Bs["MDCAID_RATE_2018BS"] + df_POPMM_2018Bs["MDCARE_RATE_2018BS"])
df_POPMM_2018Bs = df_POPMM_2018Bs[["STATE", "STNAME", "MDCAID_RATE_2018BS", "MDCARE_RATE_2018BS", "PRIVATE_RATE_2018BS"]]

In [46]:
df_POPMM_2018Bs.sort_values(by="MDCAID_RATE_2018BS")

,STATE,STNAME,MDCAID_RATE_2018BS,MDCARE_RATE_2018BS,PRIVATE_RATE_2018BS
44,UT,Utah,0.091235,0.112805,0.795960
50,WY,Wyoming,0.100596,0.174673,0.724731
34,ND,North Dakota,0.119819,0.159185,0.720995
46,VA,Virginia,0.123661,0.159901,0.716437
41,SD,South Dakota,0.125532,0.184201,0.690266
27,NE,Nebraska,0.128292,0.165617,0.706091
16,KS,Kansas,0.133792,0.170612,0.695596
29,NH,New Hampshire,0.135998,0.196414,0.667588
25,MO,Missouri,0.148749,0.187639,0.663612
43,TX,Texas,0.150117,0.131886,0.717996


### 3.2 Calculate our county-level weights for Indiana/Connecticut/New York
#### 3.2.1 Indiana County Medicaid Enrollment
- Medicaid county level: df_IN_MDcaid_2018Bc
- population county level: df_POP_allData_2018Bc
- Medicare county level: df_MDcare_2018Bc

In [47]:
def calculate_county_level_weights(df_pop, df_mdcaid, df_mdcare, statename, stateabbr):
    """
        This is the function to calculate county-level weights for each state based on their 
        medicaid, medicare, private provider insurance proportion
        args:
            -df_pop(dataframe): all data population dataframe
            -df_mdcaid(dataframe): per county-level medicaid enrollment data
            -df_mdcare(dataframe): per county-level medicare enrollment data
        returns:
    """
    df_state_pop = df_pop[df_pop["STNAME"] == statename]
    df_state_pop = df_state_pop[["STNAME", "CTYNAME", "POPESTIMATE2018"]].copy()
    
    df_state_mdcare = df_mdcare[df_mdcare["STATE"] == stateabbr]

    df_state_mdcare = df_state_mdcare[["STATE", "CTYNAME", "MDCARE_CNT_2018"]].copy()

    df_state_M1 = pd.merge(df_state_pop, df_mdcaid, on="CTYNAME")

    df_state_M2 = pd.merge(df_state_M1, df_state_mdcare, on="CTYNAME")

    df_state_MM = df_state_M2[["STNAME", "CTYNAME", "POPESTIMATE2018", "MDCAID_CNT", "MDCARE_CNT_2018"]].copy()
    # Insert new column "COUNTY_MDCAID_PT", "COUNTY_MDCARE_PT", "COUNTY_PRIVATE_PT"
    df_state_MM["MDCAID_RATE_2018BC"] = df_state_MM["MDCAID_CNT"] / df_state_MM["POPESTIMATE2018"]
    df_state_MM["MDCARE_RATE_2018BC"] = df_state_MM["MDCARE_CNT_2018"] / df_state_MM["POPESTIMATE2018"]
    df_state_MM["PP_RATE_2018BC"] = 1 - (df_state_MM["MDCAID_RATE_2018BC"]  + df_state_MM["MDCARE_RATE_2018BC"])
    df_county_weights = df_state_MM[["STNAME", "CTYNAME", "MDCAID_RATE_2018BC", "MDCARE_RATE_2018BC", "PP_RATE_2018BC"]].copy()
    
    return df_county_weights

In [48]:
df_IN_MMP_Weights_2018Bc = calculate_county_level_weights(df_POP_alldata_2018Bc, df_IN_MDcaid_2018Bc,
                                                          df_MDcare_2018Bc, "Indiana", "IN")
df_IN_MMP_Weights_2018Bc

,STNAME,CTYNAME,MDCAID_RATE_2018BC,MDCARE_RATE_2018BC,PP_RATE_2018BC
0,Indiana,Adams,0.127680,0.162336,0.709984
1,Indiana,Allen,0.215044,0.165541,0.619415
2,Indiana,Bartholomew,0.165650,0.178374,0.655976
3,Indiana,Benton,0.243153,0.205362,0.551485
4,Indiana,Blackford,0.260604,0.254987,0.484409
5,Indiana,Boone,0.100449,0.147614,0.751937
6,Indiana,Brown,0.177104,0.250886,0.572010
7,Indiana,Carroll,0.157649,0.210364,0.631987
8,Indiana,Cass,0.236359,0.202371,0.561270
9,Indiana,Clark,0.197929,0.184722,0.617348


#### 3.2.2 Connecticut
- Medicaid county level: df_CT_MDcaid_2018Bc
- population county level: df_POP_allData_2018Bc
- Medicare county level: df_MDcare_2018Bc

In [49]:
df_CT_MMP_Weights_2018Bc = calculate_county_level_weights(df_POP_alldata_2018Bc, df_CT_MDcaid_2018Bc,
                                                          df_MDcare_2018Bc, "Connecticut", "CT")
df_CT_MMP_Weights_2018Bc

,STNAME,CTYNAME,MDCAID_RATE_2018BC,MDCARE_RATE_2018BC,PP_RATE_2018BC
0,Connecticut,Fairfield,0.253588,0.148663,0.597749
1,Connecticut,Hartford,0.331112,0.176927,0.491961
2,Connecticut,Litchfield,0.245662,0.214068,0.540271
3,Connecticut,Middlesex,0.210503,0.203163,0.586334
4,Connecticut,New Haven,0.346999,0.174385,0.478616
5,Connecticut,New London,0.294422,0.190188,0.515391
6,Connecticut,Tolland,0.168307,0.161972,0.669721
7,Connecticut,Windham,0.337939,0.183154,0.478907


#### 3.2.3 New York
- Medicaid county level: df_NY_MDcaid_2018Bc
- population county level: df_POP_allData_2018Bc
- Medicare county level: df_MDcare_2018Bc

In [50]:
df_NY_MMP_Weights_2018Bc = calculate_county_level_weights(df_POP_alldata_2018Bc, df_NY_MDcaid_2018Bc,
                                                          df_MDcare_2018Bc, "New York", "NY")
df_NY_MMP_Weights_2018Bc

,STNAME,CTYNAME,MDCAID_RATE_2018BC,MDCARE_RATE_2018BC,PP_RATE_2018BC
0,New York,Albany,0.144717,0.178951,0.676332
1,New York,Allegany,0.174887,0.209757,0.615356
2,New York,Broome,0.180993,0.220704,0.598302
3,New York,Cattaraugus,0.188795,0.221707,0.589498
4,New York,Chautauqua,0.206466,0.230969,0.562565
5,New York,Chemung,0.185605,0.226185,0.588209
6,New York,Columbia,0.160642,0.240403,0.598955
7,New York,Cortland,0.167137,0.181377,0.651486
8,New York,Dutchess,0.125985,0.186441,0.687574
9,New York,Erie,0.184840,0.197124,0.618036


## 4 Output
### Compute the state-level Physician Fee Estimation and Formula
We have got a table of rate of medicaid enrollment over state population, rate of medicare beneficiaries over state population. 

The estimated physician fee = medicaid fee amount * medicaid rate + medicare fee amount * medicare rate + private provider amount * private provider rate

In [51]:
def compute_estimated_state_pfs(df_pop, df_pfs, state_abbr):
    """
        To compute the estimated physician fee adjusted on state-level
        args:
            df_pop(dataframe): state level weights dataframe
            df_pfs(dataframe): state level physician fee schedule for each insurance type (medicaid, medicare, pp)
            state_abbr: the abbreviation of the state
        returns:
            df_pfs(dataframe): new pfs
    """
    #get the index No. where the statename matches
    state_index = df_pop[df_pop["STATE"] == state_abbr].index.tolist()[0]
    #print("state_index is {}".format(state_index))
    #access state level weights=rate of each insurance type
    MDcaid_Rate = df_pop.iloc[state_index, 2]
    MDcare_Rate = df_pop.iloc[state_index, 3]
    PP_Rate = df_pop.iloc[state_index, 4]

    df_pfs["EST_PFS_AMT_BS"] = (df_pfs["MDCAID_PFS_AMT"]*MDcaid_Rate + \
                                df_pfs["MDCARE_PFS_AMT"]*MDcare_Rate + \
                                df_pfs["PP_PFS_AMT"]*PP_Rate).round(2)
    return df_pfs

In [52]:
def compute_estimated_county_pfs(df_weights, df_pfs):
    """
        To compute the estimated Physician Fee Schedule Amount for county level
        args:
            df_weights(dataframe): county level population/enrollment weights dataframe
            df_pfs(dataframe): county level physician fee schedule for each insurance type (medicaid, medicare, pp)
        returns:
            df_pfs(dataframe): new pfs
    """
    # create a county list to insert into the pfs table
    county_List = ["PFS_AMT_BY_" + "_".join(county.split(" ")) for county in df_weights["CTYNAME"]]
    # initialize attribute amounts as 0.0
    df_pfs[county_List] = pd.DataFrame([[0.0]*len(county_List)], index=df_pfs.index)

    # start computing the price based on population rate in each county
    for i, county in enumerate(county_List):
        # find out the conresponding row index in df_weights with i
        # get medicaid, medicare, private insurance population rate
        mdcaid_rate = df_weights.iloc[i, 2]
        mdcare_rate = df_weights.iloc[i, 3]
        pp_rate = df_weights.iloc[i, 4]
        # use vectorization is better than loops (Time complexity + efficiency)
        df_pfs[county] = (df_pfs["MDCAID_PFS_AMT"]*mdcaid_rate + \
                          df_pfs["MDCARE_PFS_AMT"]*mdcare_rate + \
                          df_pfs["PP_PFS_AMT"]*pp_rate).round(2)
    return df_pfs

In [53]:
def generate_output(df_pfs, state):
    """
        The function exports the physician fee schedule into CSV files for presentation
        args:
            df_pfs(dataframe): physician fee schedule dataset
            state(string): state abbreviation "CT", "IN", "NY"
    """
    df_dict = {"CT":df_CT_PFS_2018, "IN":df_IN_PFS_2018, "NY":df_NY_PFS_2018}
    filename = state + '_PFS.csv'
    filepath = './output'
    if not os.path.exists(filepath):
        os.mkdir(filepath)
    full_filename = os.path.join(filepath, filename)
    
    df_dict[state].to_csv(full_filename)

In [54]:
# compute state level estimation
df_IN_PFS_2018 = compute_estimated_state_pfs(df_POPMM_2018Bs, df_IN_PFS_2018, "IN")
# compute county level estimation
df_IN_PFS_2018 = compute_estimated_county_pfs(df_IN_MMP_Weights_2018Bc, df_IN_PFS_2018)
# Generate output for state Indiana
generate_output(df_IN_PFS_2018, "IN")
df_IN_PFS_2018.head()

,PROC_CODE,SHORT DESCRIPTION,MDCAID_PFS_AMT,MDCARE_PFS_AMT,PP_PFS_AMT,EST_PFS_AMT_BS,PFS_AMT_BY_Adams,PFS_AMT_BY_Allen,PFS_AMT_BY_Bartholomew,PFS_AMT_BY_Benton,...,PFS_AMT_BY_Vermillion,PFS_AMT_BY_Vigo,PFS_AMT_BY_Wabash,PFS_AMT_BY_Warren,PFS_AMT_BY_Warrick,PFS_AMT_BY_Washington,PFS_AMT_BY_Wayne,PFS_AMT_BY_Wells,PFS_AMT_BY_White,PFS_AMT_BY_Whitley
0,10030,Guide cathet fluid drainage,539.11,533.82,533.82,534.97,534.50,534.96,534.70,535.11,...,535.15,535.17,534.91,534.74,534.52,535.03,535.23,534.61,534.87,534.47
1,10035,Perq dev soft tiss 1st imag,480.64,488.31,488.31,486.65,487.33,486.66,487.04,486.45,...,486.38,486.35,486.72,486.97,487.30,486.55,486.27,487.17,486.79,487.37
2,19081,Bx breast 1st lesion strtctc,702.08,652.03,652.03,662.88,658.42,662.79,660.32,664.20,...,664.61,664.81,662.39,660.77,658.63,663.49,665.37,659.49,661.96,658.16
3,19083,Bx breast 1st lesion us imag,702.08,634.19,634.19,648.91,642.86,648.79,645.44,650.70,...,651.26,651.52,648.24,646.05,643.14,649.73,652.28,644.31,647.67,642.50
4,19085,Bx breast 1st lesion mr imag,702.08,948.13,948.13,894.78,916.71,895.22,907.37,888.30,...,886.28,885.31,897.22,905.16,915.70,891.81,882.56,911.47,899.29,918.01


In [55]:
df_CT_PFS_2018 = compute_estimated_state_pfs(df_POPMM_2018Bs, df_CT_PFS_2018, "CT")
df_CT_PFS_2018 = compute_estimated_county_pfs(df_CT_MMP_Weights_2018Bc, df_CT_PFS_2018)
# Generate output for state Connecticut
generate_output(df_CT_PFS_2018, "CT")
df_CT_PFS_2018.head()

,PROC_CODE,SHORT DESCRIPTION,MDCAID_PFS_AMT,MDCARE_PFS_AMT,PP_PFS_AMT,EST_PFS_AMT_BS,PFS_AMT_BY_Fairfield,PFS_AMT_BY_Hartford,PFS_AMT_BY_Litchfield,PFS_AMT_BY_Middlesex,PFS_AMT_BY_New_Haven,PFS_AMT_BY_New_London,PFS_AMT_BY_Tolland,PFS_AMT_BY_Windham
0,10121,Remove foreign body,446.00,306.48,306.48,339.91,341.86,352.68,340.75,335.85,354.89,347.56,329.96,353.63
1,10180,Complex drainage wound,446.00,277.66,277.66,317.99,320.35,333.40,319.01,313.10,336.07,327.22,305.99,334.55
2,11010,Debride skin at fx site,251.52,568.10,568.10,492.25,487.82,463.28,490.33,501.46,458.25,474.89,514.82,461.12
3,11011,Debride skin musc at fx site,251.52,608.45,608.45,522.94,517.94,490.27,520.77,533.32,484.60,503.36,548.38,487.83
4,11012,Deb skin bone at fx site,251.52,795.03,795.03,664.82,657.20,615.07,661.51,680.62,606.43,635.01,703.55,611.36


In [56]:
df_NY_PFS_2018 = compute_estimated_state_pfs(df_POPMM_2018Bs, df_NY_PFS_2018, "NY")
df_NY_PFS_2018 = compute_estimated_county_pfs(df_NY_MMP_Weights_2018Bc, df_NY_PFS_2018)
# Generate output for state New York
generate_output(df_NY_PFS_2018, "NY")
df_NY_PFS_2018.head()

,PROC_CODE,SHORT DESCRIPTION,MDCAID_PFS_AMT,MDCARE_PFS_AMT,PP_PFS_AMT,EST_PFS_AMT_BS,PFS_AMT_BY_Albany,PFS_AMT_BY_Allegany,PFS_AMT_BY_Broome,PFS_AMT_BY_Cattaraugus,...,PFS_AMT_BY_Steuben,PFS_AMT_BY_Suffolk,PFS_AMT_BY_Sullivan,PFS_AMT_BY_Tioga,PFS_AMT_BY_Tompkins,PFS_AMT_BY_Ulster,PFS_AMT_BY_Warren,PFS_AMT_BY_Washington,PFS_AMT_BY_Wayne,PFS_AMT_BY_Westchester
0,10021,Fna w/o image,81.61,118.79,118.79,106.97,113.41,112.29,112.06,111.77,...,112.19,113.39,109.67,112.99,114.93,112.80,113.16,112.28,112.63,113.43
1,10030,Guide cathet fluid drainage,504.97,550.32,550.32,535.90,543.76,542.39,542.11,541.76,...,542.27,543.74,539.20,543.24,545.61,543.01,543.45,542.38,542.81,543.79
2,10035,Perq dev soft tiss 1st imag,349.63,503.96,503.96,454.90,481.63,476.97,476.03,474.82,...,476.56,481.56,466.10,479.87,487.93,479.10,480.58,476.95,478.40,481.73
3,10036,Perq dev soft tiss add imag,304.15,441.81,441.81,398.05,421.89,417.74,416.89,415.82,...,417.37,421.83,408.04,420.32,427.51,419.63,420.96,417.72,419.01,421.98
4,10040,Acne surgery,54.25,107.02,107.02,90.24,99.38,97.79,97.47,97.06,...,97.65,99.36,94.08,98.78,101.54,98.52,99.03,97.79,98.28,99.42


In [57]:
maxFee = 0
for i in range(0, len(df_NY_PFS_2018["PROC_CODE"])):
#     print("max is {}, min is {}".format(df_NY_PFS_2018.iloc[i, 5:52].max(), df_NY_PFS_2018.iloc[i, 5:52].min()))
    maxFee = max(maxFee, df_NY_PFS_2018.iloc[i, 5:52].max() - df_NY_PFS_2018.iloc[i, 5:52].min())
maxFee

2551.300000000001